## 정리노트 #3

DeZero의 성능을 개선하기 위해 처리 속도와 메모리 사용량을 개선한다.

파이썬의 메모리 관리는 두 가지 방식이 있는데, 참조(reference) 수를 세는 방식과 Garbage Collection 방식(세대를 기준으로 쓸모없어진 객체를 회수하는 방식)이 있다. 참조 카운트는 파이썬 메모리 관리의 기본이고 구조가 간단하며 속도도 빠르다.

*참조 카운트* : 
모든 객체는 참조 카운트가 0인 상태로 생성되고 다른 객체가 참조할 때 마다 1씩 증가, 객체에 대한 참조가 끊길 때 마다 1씩 감소. 0이 되면 파이썬 인터프리터가 회수

참조 카운트는 순환참조 문제를 해결할 수 없다. 순환 참조는 객체가 원 모양을 그리며 서로가 서로를 참조하게 되는 상태인데, 파이썬 모듈인 weakref로 해결할 수 있다.

In [1]:
import weakref # 약한 참조를 만들기(참조는 하되 카운트를 증가시키지 않는다)
import numpy as np

a = np.array([1,2,3])
b = weakref.ref(a)

In [2]:
b

<weakref at 0x000001F3A9520E50; to 'numpy.ndarray' at 0x000001F3A94B86F0>

In [3]:
b() # 데이터에 접근

array([1, 2, 3])

In [4]:
a = None
b 

<weakref at 0x000001F3A9520E50; to 'numpy.ndarray' at 0x000001F3A94B86F0>

IPython과 주피터 노트북 등의 인터프리터는 인터프리터 자체가 모르는 참조를 추가로 유지하기 때문에 앞의 코드에서 b가 여전히 유효한 참조를 유지한다. 위 코드는 파이썬 인터프리터에서 실행하면 정상적으로 dead 된다 

In [5]:
'''DeZero에 weakref구조 도입'''

import weakref

class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs] 
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple): 
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)        
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs] # 인스턴스 변수 self.outputs가 대상을 약한 참조로 가리키게 변경
        return outputs if len(outputs) > 1 else outputs[0]

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1 
        return y

In [6]:
'''backward 메서드 수정'''

class Variable:
    def __init__(self,data):
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
                                                       
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            # gys = [output.grad for output in f.outputs] : 수정 전
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
   
    def cleargrad(self):
        self.grad = None

In [7]:
# 동작 확인을 위해 note#2의 코드를 가져옴

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data 
        gx = 2 * x * gy
        return gx 
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self,gy): 
        return gy, gy 
    
def square(x):
    f = Square()
    return f(x)

def add(x0, x1):
    return Add()(x0, x1)

def as_array(x):
    if np.isscalar(x):  
        return np.array(x)  
    return x

In [8]:
'''동작 확인'''

for i in range(10):
    x = Variable(np.random.randn(100000)) # 거대한 데이터
    y = square(square(square(x))) # 복잡한 계산을 수행한다

파이썬의 메모리 사용량을 측정하려면 외부 라이브러리인 memory profiler 등을 사용하면 편리하다!

#### DeZero의 메모리 사용을 개선할 수 있는 구조 두 가지 
1. 역전파 시 사용하는 메모리양을 줄이는 방법 (불필요한 미분 결과를 보관하지 않고 즉시 삭제)
2. '역전파가 필요 없는 경우'용 모드를 제공 (이 모드에서는 불필요한 계산을 생략)

In [9]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad) 

1.0 1.0
2.0 1.0


위 결과를 보면, 현재의 DeZero는 모든 변수가 미분값을 변수에 저장해 두고 있다.

하지만 대부분 역전파로 구하고 싶은 미분값은 말단 변수(x0, x1)이고 y와 t같은 중간 변수의 미분값은 필요하지 않다.

중간 변수에 대해서는 미분값을 제거하는 코드를 추가하여 메모리 사용에 관해 개선한다. 

In [10]:
'''첫 번째 개선 = 필요 없는 미분값 삭제'''

class Variable:
    def __init__(self,data):
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False): #메서드의 인수에 retain_grad를 추가 : 중간 변수의 미분값을 모두 None으로 재설정
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
                                                       
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None # 각 함수의 출력 변수의 미분값을 유지하지 않도록 설정 / y는 약한 참조
   
    def cleargrad(self):
        self.grad = None

In [11]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad) 

None None
2.0 1.0


위와 같이 중간 변수인 y와 t의 미분값이 삭제되어 그만큼의 메모리를 다른 용도로 사용할 수 있다.

신경망에는 **학습**과 **추론**이라는 두 가지 단계가 있는데, 학습 시에는 미분값을 구해야 하지만 추론 시에는 단순히 순전파만 하기 때문에 중간 계산 결과를 곧바로 버리면 메모리 사용량을 크게 줄일 수 있다.

미분값이 필요 없는 경우 중간 계산 결과를 저장할 필요가 없기 때문에 순전파만 할 경우를 위한 개선을 Dezero에 추가한다.

In [12]:
'''두 번째 개선 = 순전파만 할 경우를 위한 개선 / 역전파 활성 모드와 역전파 비활성 모드를 전환하는 구조 필요'''

class Config:
    enable_backprop = True # 역전파가 가능한지 여부를 뜻하고 이 값이 True면 역전파 활성 모드이다.

In [13]:
'''Function에서 config 클래스를 참조하여 모드를 전환하게 함'''

class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs] 
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple): 
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop: # 코드 추가
            self.generation = max([x.generation for x in inputs]) # 세대 설정
            for output in outputs:
                output.set_creator(self)  # 연결 설정      
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

#### 파이썬의 with문 : 후처리를 자동으로 수행하고자 할 때 사용
with문을 사용하는 것으로 with블록에 들어갈 때의 처리(전처리)와 with 블록을 빠져나올 때의 처리(후처리)를 자동으로 할 수 있다. 


In [14]:
'''with문을 사용하여 역전파 비활성 모드로 전환 - contextlib 모듈'''

import contextlib

@contextlib.contextmanager # 문맥을 판단하는 함수가 만들어짐
def using_config(name, value):
    old_value = getattr(Config, name) 
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

▶ **getattr 함수**는 Python에서 객체의 속성값을 가져오는 내장 함수. Config 클래스에서 name 매개변수에 해당하는 속성의 현재 값을 old_value에 저장, 설정을 원래대로 복구하기 위해 필요.

▶ **setattr 함수**는 지정된 객체에 새로운 속성을 추가하거나 기존 속성의 값을 변경. 위 코드에선 Config의 name 설정을 value로 변경.

▶**yield 문**은 with 블록 내의 코드가 실행되는 지점을 나타냄.

▶ **finally 블록**에서 with 블록을 떠나기 전 setattr(Config, name, old_value)를 사용하여 설정을 원래의 값(old_value)으로 복원.

이 함수를 사용하여 해당 블록 내에서만 설정을 변경할 수 있다.

In [15]:
def no_grad(): # 기울기가 필요 없을 때는 no_grad 함수를 호출 / 순전파 계산만 필요할 때 '모드 전환'을 사용
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

In [16]:
'''변수 사용성 개선 - 변수들을 서로 구분하기 위해 이름을 붙여줄 수 있도록 설정'''

class Variable:
    def __init__(self,data, name=None): # name이라는 인스턴스 변수를 추가
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
                                                       
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 

사용하는 사람 입장에서 데이터를 담는 상자보다 그 안의 데이터가 더 중요하다. 아래 코드는 Variable 인스턴스를 ndarray 인스턴스처럼 보이게 해 주는 장치를 추가한 코드이다.  

In [17]:
'''ndarray 인스턴스 변수'''

class Variable:
    def __init__(self,data, name=None): 
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
    
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 
                    
    @property # 메서드 추가
    def shape(self):
        return self.data.shape
    
    @property # 메서드 추가
    def ndim(self):
        return self.data.ndim
    
    @property # 메서드 추가
    def size(self):
        return self.data.size
 
    @property # 메서드 추가
    def dtype(self):
        return self.data.dtype

인스턴스 변수 shape은 다차원 배열의 형상을 알려주고, ndim은 차원 수, size는 원소 수, dtype은 데이터 타입을 나타낸다. 

@propety 데코레이터는 파이썬에서 메서드를 속성처럼 접근할 수 있게 해 주는 구문. 위 코드에 추가되어 메서드를 인스턴스 변수처럼 사용할 수 있다.

In [18]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(x.shape) # x.shape()대신 x.shape로 호출할 수 있다. 괄호를 사용하지 않고도 해당 메서드의 값을 속성처럼 쉽게 가져올 수 있다.

(2, 3)


파이썬의 **len 함수**는 객체 수를 알려주는 파이썬의 표준 함수이다. 리스트 등에 사용하면 그 안에 포함된 원소 수를 반환한다.

In [19]:
'''len 함수가 Variable 안의 원소 수를 인식'''

class Variable:
    def __init__(self,data, name=None): 
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
    
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 
                    
    @property 
    def shape(self):
        return self.data.shape
    
    @property 
    def ndim(self):
        return self.data.ndim
    
    @property 
    def size(self):
        return self.data.size
 
    @property 
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): # Variable 인스턴스에 대해 len 함수를 사용
        return len(self.data)

파이썬에서 '\_\_init\_\_'과 '\_\_len\_\_'등 특별한 의미를 지닌 메서드는 밑줄 두개로 감싼 이름을 사용한다.

In [20]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(len(x))

2


In [21]:
'''print 함수를 사용하여 Variable 안의 데이터 내용을 출력'''

class Variable:
    def __init__(self,data, name=None): 
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
    
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 
                    
    @property 
    def shape(self):
        return self.data.shape
    
    @property 
    def ndim(self):
        return self.data.ndim
    
    @property 
    def size(self):
        return self.data.size
 
    @property 
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'Variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9) # ndarray 인스턴스를 문자열로 반환, 줄바꿈이 있으면 바꾼 후 공백을 출력하여 시작 위치를 가지런하게 만듦
        return 'variable(' + p + ')' # 변환된 문자열 p를 'variable(...)' 형태의 문자열로 감싸서 반환

\_\_repr__ 메서드는 객체를 대표하는 문자열을 반환하는데, 주로 개발자가 디버깅을 위해 그 객체의 정보를 문자열 형태로 보고 싶을 때 사용

#### 연산자 오버로드

In [22]:
'''Mul 클래스 구현'''

class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data # 해당 연산에 대한 입력값들을 의미
        return gy * x1, gy * x0
    
# Mul 클래스를 파이썬 함수로 사용    
def mul(x0, x1):
    return Mul()(x0, x1)

In [23]:
'''mul 함수를 이용하여 곱셈 하기'''

a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a,b),c)
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


위 코드와 같이 매번 y = add(mul(a,b),c)처럼 코딩하기 번거롭기 때문에 y = a * b + c 형태로 바꿔준다.

**연산자를 오버로드**하면 +와 \*같은 연산자 사용 시 사용자가 설정한 함수가 호출된다.

In [24]:
'''곱셈 연산자 * 오버로드'''

class Variable:
    def __init__(self,data, name=None): 
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
    
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 
                    
    @property 
    def shape(self):
        return self.data.shape
    
    @property 
    def ndim(self):
        return self.data.ndim
    
    @property 
    def size(self):
        return self.data.size
 
    @property 
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'Variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
    
    def __mul__(self, other):
        return mul(self, other) # __mul__ 메서드 정의

In [25]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))

y = a * b

print(y)

variable(6.0)


a * b가 실행될 때 인스턴스 a의 \_\_mul__(self, other) 메서드가 호출된다. a는 인수 self에 전달되고 b가 other에 전달된다.

+) 인스턴스 a의 \_\_mul__(self, other) 메서드인 이유는 * 연산자를 사용하면 파이썬의 데이터 모델에 따라 먼저 왼쪽 피연산자의 __mul__ 메서드를 호출하려고 시도하여 파이썬은 a.__mul__(b)를 호출하게 된다. 연산자의 왼쪽 피연산자가 해당 메서드를 구현하고 있지 않거나 연산을 지원하지 않을 때만 오른쪽 피연산자의 반대 연산자 오버로딩 메서드(예: __rmul__)를 시도한다

In [26]:
'''간단한 코드로 변경'''

class Variable:
    def __init__(self,data, name=None): 
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
    
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 
                    
    @property 
    def shape(self):
        return self.data.shape
    
    @property 
    def ndim(self):
        return self.data.ndim
    
    @property 
    def size(self):
        return self.data.size
 
    @property 
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'Variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
    
Variable.__mul__ = mul
Variable.__add__ = add # 함수 자체를 할당할 수 있다

In [27]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a,b),c)
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


Variable 인스턴스와 ndarray 인스턴스, int나 float등 함께 사용할 수 있도록 구현하기 위해서 a가 Variable 인스턴스일 때 
a * np.array(2.0)이라는 코드를 만나면 ndarray 인스턴스를 자동으로 Variable 인스턴스로 변환하게 구현한다.

as_variable이라는 함수를 준비하여 인수로 주어진 객체를 Variable 인스턴스로 변환하게 한다.

In [28]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj # obj가 Variable 인스턴스면 아무것도 손보지 않고 그대로 반환
    return Variable(obj) # obj가 Variable 인스턴스가 아니라면 Variable 인스턴스로 변환하여 반환

In [29]:
'''간단한 코드로 변경'''
import numpy as np
import weakref

class Variable:
    def __init__(self,data, name=None): 
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
    
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 
                    
    @property 
    def shape(self):
        return self.data.shape
    
    @property 
    def ndim(self):
        return self.data.ndim
    
    @property 
    def size(self):
        return self.data.size
 
    @property 
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'Variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
    
# 오류로 인한 코드 추가 
def variable_add(self, other):
    other = as_variable(other)
    return add(self, other)

def variable_mul(self, other):
    other = as_variable(other)
    return mul(self, other)

Variable.__add__ = variable_add
Variable.__mul__ = variable_mul

In [30]:
'''Function에서 __call 메서드가 as_variable 함수를 이용하도록 코드 추가'''

class Function:
    def __call__(self, *inputs): 
        inputs = [as_variable(x) for x in inputs] # 코드 추가       
        xs = [x.data for x in inputs] 
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple): 
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop: 
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)   
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

In [31]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


### 오류
코드를 실행하니 AttributeError: 'numpy.ndarray' object has no attribute 'generation'이라는 오류가 발생해서 Variable 클래스에 

def variable_add(self, other):
    other = as_variable(other)
    return add(self, other)

def variable_mul(self, other):
    other = as_variable(other)
    return mul(self, other)
    
코드를 추가해 주었다.

In [32]:
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

In [33]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

variable(5.0)


### 현재 코드의 문제점
1. **첫 번째 인수가 float나 int인 경우** : 2.0 * x를 실행했을 때 연산자 왼쪽에 있는 2.0의 \_\_mul__메서드를 호출하려 시도하지만, 2.0의 \_\_mul__메서드는 구현되어 있지 않다. 따라서 오른쪽에 있는 x의 특수 메서드인 \_\_rmul__메서드를 호출하려 시도하지만, Variable 클래스에는 \_\_rmul__메서드가 구현되어 있지 않다. 따라서 **\_\_rmul__메서드를 구현하면 해결**된다.
2. **좌항이 ndarray 인스턴스인 경우** : 좌항인 ndarray 인스턴스의 \_\_add__메서드가 호출되지만 우항인 인스턴스의 \_\_radd__ 메서드가 실행되길 바라는 경우, **연산자 우선순위**를 지정해 주어 우선적으로 호출될 수 있게 한다.  

In [34]:
class Variable:
    def __init__(self,data, name=None): 
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
    
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 
                    
    @property 
    def shape(self):
        return self.data.shape
    
    @property 
    def ndim(self):
        return self.data.ndim
    
    @property 
    def size(self):
        return self.data.size
 
    @property 
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'Variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
    
def variable_add(self, other):
    other = as_variable(other)
    return add(self, other)

def variable_mul(self, other):
    other = as_variable(other)
    return mul(self, other)

# 곱셈과 덧셈에서는 좌항과 우항을 바꾸어도 결과가 동일하다.
Variable.__add__ = variable_add
Variable.__radd__ = variable_add
Variable.__mul__ = variable_mul 
Variable.__rmul__ = variable_mul

In [35]:
x = Variable(np.array(2.0))
y = 3.0 * x + 1.0
print(y)

variable(7.0)


In [36]:
import numpy as np
import weakref

class Variable:
    __array_priority__ = 200 #  NumPy의 배열 연산에서 우선 순위를 결정하는 속성
    def __init__(self,data, name=None): 
                
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0 
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 
        
    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 
    
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)
                
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance (gxs, tuple): 
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
                
            if not retain_grad: 
                for y in f.outputs:
                    y().grad = None 
                    
    @property 
    def shape(self):
        return self.data.shape
    
    @property 
    def ndim(self):
        return self.data.ndim
    
    @property 
    def size(self):
        return self.data.size
 
    @property 
    def dtype(self):
        return self.data.dtype
    
    def __len__(self): 
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'Variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
    
def variable_add(self, other):
    other = as_variable(other)
    return add(self, other)

def variable_mul(self, other):
    other = as_variable(other)
    return mul(self, other)

Variable.__add__ = variable_add
Variable.__radd__ = variable_add
Variable.__mul__ = variable_mul 
Variable.__rmul__ = variable_mul

#### 새로운 연산자를 추가하는 순서
1. Function 클래스를 상속하여 원하는 함수 클래스를 구현한다.
2. 파이썬 함수로 사용할 수 있도록 한다.
3. Variable 클래스의 연산자를 오버로드한다.

**\_\_neg__(self)** 는 양수를 음수로, 음수를 양수로 바꿔주는 부호 변환 연산자로 항이 하나뿐인 단항 연산자이다.

**\_\_sub__(self, other)** 는 뺄셈 연산자로 이항 연산자이다.

**\_\_truediv__(self, other)** 는 나눗셈 연산자로 이항 연산자이다.

**\_\_pow__(self, other)** 는 거듭제곱 연산자로 이항 연산자이다. 코드에선 좌항이 Variable 인스턴스이고 우항이 상수인 경우만을 고려하였다.


In [37]:
'''연산자 추가'''

class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy
    
def neg(x):
    return Neg()(x) # Neg 인스턴스를 생성하고, 이 인스턴스에 대해 x를 인자로 하여 호출

Variable.__neg__ = neg

In [38]:
x = Variable(np.array(2.0))
y = -x
print(y)

variable(-2.0)


In [39]:
'''연산자 추가'''

class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y
    
    def backward(self, gy):
        return gy, -gy
    
def sub(x0,x1):
    x1 = as_array(x1)
    return Sub()(x0,x1)

Variable.__sub__ = sub

뺄셈은 우항과 좌항의 순서를 바꾸면 값이 달라지기 때문에 좌우를 구별해 주어야 한다. 우항을 대상으로 했을 때 적용할 함수인 rsub(x0,x1)을 별도로 준비해야 한다.

In [40]:
'''연산자 추가'''

class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y
    
    def backward(self, gy):
        return gy, -gy
    
def sub(x0,x1):
    x1 = as_array(x1)
    return Sub()(x0,x1)

def rsub(x0,x1):
    x1 = as_array(x1)
    return Sub()(x1,x0) # x0과 x1의 순서를 바꿈

Variable.__sub__ = sub
Variable.__rsub__ = rsub

In [41]:
x = Variable(np.array(2.0))
y1 = 2.0 - x
y2 = x - 1.0
print(y1)
print(y2)

variable(0.0)
variable(1.0)


In [42]:
'''연산자 추가'''

class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)


def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0)

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [43]:
'''연산자 추가'''

class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow

거듭제곱은 Pow 클래스를 초기화 할 때 지수 c를 제공할 수 있다. 순전파 메서드인 forward는(x)는 밑에 해당하는 x만 받게 한다.

In [44]:
y = x ** 3
y.backward()
print(y)

variable(8.0)


### Dezero를 패키지로 정리
- 모듈 : 파이썬 파일로 다른 프로그램에서 import하여 사용하는 것을 가정하고 만들어진 것이다.
- 패키지 : 여러 모듈을 묶은 것으로 패키지를 만드려면 먼저 디렉터리를 만들고 그 안에 모듈(파이썬 파일)을 추가해야 한다.
- 라이브러리 : 여러 패키지를 묶은 것으로 하나 이상의 디렉터리로 구성된다. 때로는 패키지를 라이브러리라고 부른다.

코드를 dezero/core_simple.py라는 코어 파일로 옮기고 외부의 파이썬 파일에서 dezero를 임포트한다.

In [45]:
import numpy as np
from dezero.core_simple import Variable

x = Variable(np.array(1.0))
print(x)

variable(1.0)


In [46]:
import numpy as np
from dezero.core_simple import Variable

x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)

variable(16.0)
8.0


In [48]:
'''
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

import numpy as np
from dezero import Variable


x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)'''

import os
import sys

# 현재 작업 디렉토리를 sys.path에 추가
current_dir = os.getcwd()
sys.path.append(current_dir)

import numpy as np
from dezero import Variable

x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)

variable(16.0)
8.0


주피터 노트북 환경에서는 \_\_file__ 전역 변수가 정의되어 있지 않기 때문에, 현재 작업 디렉토리를 기준으로 모듈 경로를 추가했다. os.getcwd()를 사용하여 현재 작업 디렉토리를 얻고, 이를 기반으로 모듈 경로를 추가하는 방식을 사용했다.